In [49]:
import seaborn as sns
import pandas as pd;

sns.set(rc={'figure.figsize':(10,8)}, font_scale = 1.3)
data = pd.read_csv('SupplFileD1_subset.csv')
orig = pd.read_csv('SupplFileD1a.csv')

In [52]:
from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *
data.Gender = data.Gender.replace('male', 0)
data.Gender = data.Gender.replace('female', 0)
data.Gender = data.Gender.replace(np.nan, 1)
data = data.dropna(subset=['Weight'])
mean=data['Height'].mean()
#print(data['HIV status'].unique())
data['Height'].fillna(mean, inplace=True)
data['BMI']=data['Weight']/((data['Height']/100)*(data['Height']/100))
data['Socio Economic Status'] = data['Socio Economic Status'].replace('APL', 1)
data['Socio Economic Status'] = data['Socio Economic Status'].replace('BPL', 2)
data['Socio Economic Status'] = data['Socio Economic Status'].replace('UNKNOWN', 3)
data['Socio Economic Status'] = data['Socio Economic Status'].replace('Unknown', 3)
data['Socio Economic Status'] = data['Socio Economic Status'].replace('No records Found', 3)
data['Socio Economic Status'] = data['Socio Economic Status'].replace('No Records Found', 3)
data['New or Previously treated'] = data['New or Previously treated'].replace('NEW', 1)
data['New or Previously treated'] = data['New or Previously treated'].replace('New', 1)
data['New or Previously treated'] = data['New or Previously treated'].replace('PREVIOUSLY TREATED', 2)
data['New or Previously treated'] = data['New or Previously treated'].replace('', 3)
data['New or Previously treated'] = data['New or Previously treated'].replace(np.nan, 3)

data['TREATMENT PHASE'] = data['TREATMENT PHASE'].replace('IP', 1)
data['TREATMENT PHASE'] = data['TREATMENT PHASE'].replace('CP', 2)
data['TREATMENT PHASE'] = data['TREATMENT PHASE'].replace('', 3)
data['TREATMENT PHASE'] = data['TREATMENT PHASE'].replace(np.nan, 3)

data['Treatment regimen'].replace(['All oral longer regimen (MDR+/- FQ or SLI)','Regimen for H Mono Poly','Retreatment: Recurrent',
                                   'Shorter MDR-TB regimen',r'^\s+$',np.nan],[1,2,3,4,5,5],inplace=True)


data['Treatment outcome'].replace(['DIED', 'CURED','TREATMENT COMPLETE','PATIENT_REFUSED','Lost to follow up',
                                'NOT_EVALUATED','ON TREATMENT','UNTRACEABLE_INCORRECT_ADDRESS',
                                'TREATMENT REGIMEN CHANGED','WRONGLY_DIAGNOSED','UNTRACEABLE_MIGRATED','TREATMENT_FAILURE','',r'^\s+$',np.nan],[0, 1,2,3,4,5,6,7,8,9,10,11,12,12,12], inplace=True)
data['Diabetes'].replace(['Non-diabetic','Diabetic','UNKNOWN','',np.nan],[1,2,3,3,3], inplace=True)
data['TOBACCO-Smoking'].replace(['YES','NO','NO/ POS-SMOKELESS','POS-SMOKELESS','UNKNOWN','',np.nan],[1,2,3,4,5,5,5], inplace=True)
data['Alcohol'].replace(['YES','NO','UNKNOWN','',np.nan],[1,2,3,3,3],inplace=True)
data['HIV status'].replace(['Non Reactive / Negative','REACTIVE','UNKNOWN','',np.nan],[1,2,3,3,3],inplace=True)
data['Rif Resistance'].replace(['NOT DETECTED','DETECTED','INVALID','UNKNOWN','','-',np.nan],[1,2,3,4,4,4,4],inplace=True)
data['Smear'].replace(['1+positive','2+positive','3+positive','Positive','Negative/Not Seen','',np.nan],[1,2,3,4,5,6,6],inplace=True)


data['RIF(D,ND,I)'].replace(['Detected','Not detected',r'^\s+$',np.nan],[1,2,3,3],regex=True,inplace=True)
data['INH(InhA) (D,ND,I)'].replace(['Detected','Not detected',r'^\s+$',np.nan],[1,2,3,3],regex=True,inplace=True)
data['INH(KatG) (D,ND,I)'].replace(['Detected','Not detected',r'^\s+$',np.nan],[1,2,3,3],inplace=True,regex=True)
data['FQ class resistance (ND/D/I)'].replace(['Detected','Not detected',r'^\s+$',np.nan],[1,2,3,3],inplace=True,regex=True)
data['SLI (resi)'].replace(['Detected','Not detected',r'^\s+$',np.nan],[1,2,3,3],inplace=True,regex=True)
data['SLID (eis)'].replace(['Not detected',r'^\s+$',np.nan],[1,2,2],inplace=True,regex=True)

data['Treatment outcome'].replace(['On treatment'],[6], inplace=True)
data['Weight'] = data['Weight'].astype(int)
#print(data[data['Height']=='-'])
data['Height'] = data['Height'].astype(int)
data['Treatment outcome']=data['Treatment outcome'].astype(int)

print(data.columns)
pd.DataFrame(data).to_csv(r'data_imputed.csv',index=False)


Index(['PT_ID', 'Age', 'Gender', 'Socio Economic Status',
       'New or Previously treated', 'Weight', 'Height', 'Treatment regimen',
       'TREATMENT PHASE', 'Treatment outcome', 'HIV status', 'Diabetes',
       'TOBACCO-Smoking', 'Alcohol', 'Rif Resistance', 'Smear', 'RIF(D,ND,I)',
       'INH(InhA) (D,ND,I)', 'INH(KatG) (D,ND,I)', 'Test Interpretation',
       'Clinical Interpretation', 'FQ class resistance (ND/D/I)', 'SLI (resi)',
       'SLID (eis)', 'Test Interpretation.1', 'COVID Result', 'BMI'],
      dtype='object')


In [32]:
np.random.RandomState(seed=20722250)
data['COVID Result'] = data['COVID Result'].replace('Positive', 1)
data['COVID Result'] = data['COVID Result'].replace('Negative', 0)
psm = PsmPy(data, treatment='COVID Result', indx='PT_ID', exclude = ['Height','Weight','Test Interpretation','Clinical Interpretation', 'Test Interpretation.1'])

In [33]:
import random
np.random.seed(seed=20722250)
psm.logistic_ps(balance = True)
psm.knn_matched(matcher='propensity_score', replacement=False, caliper=None)
Unique_num_Avg_height=orig[(pd.isna(orig['Height']))& (orig['COVID Result']=='Positive')]['PT_ID']
#psm.predicted_data['propensity_score'].values
random.seed(231122)
Unique_num_Avg_height_random2=random.sample(list(Unique_num_Avg_height), 2)
Unique_num_Avg_height_random2


/data/users/CS18D017/miniconda3/lib/python3.8/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


['PT308', 'PT043']

In [34]:
#match=list(psm.matched_ids['matched_ID'])
#match.extend(psm.matched_ids['PT_ID'])
#All those samples with height attribute in the original data were chosen (22) and remaining 2 were picked in random
import random
Unique_num=list(orig[(orig['Height']>0) & (orig['COVID Result']=='Positive')]['PT_ID'])
Unique_num_Avg_height=orig[(pd.isna(orig['Height']))& (orig['COVID Result']=='Positive')]['PT_ID']
random.seed(231122)   #date is the seed
#random.seed(240425)   #date is the seed
Unique_num_Avg_height_random2=random.sample(list(Unique_num_Avg_height), 2)
Unique_num.extend(Unique_num_Avg_height_random2)

match=list(psm.matched_ids[psm.matched_ids['PT_ID'].isin(Unique_num)]['PT_ID'])
match.extend(psm.matched_ids[psm.matched_ids['PT_ID'].isin(Unique_num)]['matched_ID'])
print(len(match),len(Unique_num))
subset1=orig[orig['PT_ID'].isin(match)]
pd.DataFrame(orig[orig['PT_ID'].isin(match)]).to_csv(r'Subset_19_11.csv',index=False) 


48 26


In [35]:
#match=list(psm.matched_ids['matched_ID'])
#match.extend(psm.matched_ids['PT_ID'])
#All those samples with height attribute in the original data were chosen (22) and remaining 2 were picked in random
import random
Unique_num=list(orig[(orig['Height']>0) & (orig['COVID Result']=='Positive')]['PT_ID'])
Unique_num_Avg_height=orig[(pd.isna(orig['Height']))& (orig['COVID Result']=='Positive')]['PT_ID']
random.seed(231122)   #date is the seed
#random.seed(500078)   #date is the seed
Unique_num_Avg_height_random2=random.sample(list(Unique_num_Avg_height), 2)
Unique_num.extend(Unique_num_Avg_height_random2)

match=list(psm.matched_ids[psm.matched_ids['PT_ID'].isin(Unique_num)]['PT_ID'])
match.extend(psm.matched_ids[psm.matched_ids['PT_ID'].isin(Unique_num)]['matched_ID'])
print(len(match),len(Unique_num))
subset=orig[orig['PT_ID'].isin(match)]
print(subset)
pd.DataFrame(orig[orig['PT_ID'].isin(match)]).to_csv(r'Subset_19_11.csv',index=False) 


48 26
     PT_ID  Age  Gender Socio Economic Status New or Previously treated  \
0    PT408   80  female                   BPL                       NEW   
4    PT014   43    male                   BPL                       NEW   
8    PT083   49    male                   BPL                       NEW   
15   PT168   45    male                   BPL                       NEW   
16   PT292   43    male               Unknown                       NEW   
43   PT313   75    male               UNKNOWN                       NEW   
44   PT319   55    male                   BPL                       NEW   
57   PT174   52    male                   BPL                       NEW   
58   PT078   50    male                   BPL                       NEW   
61   PT388   60    male                   BPL                       NEW   
62   PT051   61    male                   BPL                       NEW   
67   PT282   50  female               UNKNOWN                       NEW   
69   PT121   50  fe

In [11]:
subset[subset['COVID Result']=="Positive"].describe()


,Age,Weight,Height
count,24.000000,24.000000,22.000000
mean,51.958333,46.875000,158.727273
std,12.484701,9.900867,6.762514
min,25.000000,27.000000,144.000000
25%,46.000000,39.500000,156.000000
50%,51.500000,48.000000,160.000000
75%,58.500000,52.500000,162.750000
max,80.000000,65.000000,168.000000


In [12]:
subset[subset['COVID Result']=="Negative"].describe()

,Age,Weight,Height
count,24.000000,24.000000,16.00000
mean,55.250000,42.875000,159.18750
std,10.787473,8.461229,8.01015
min,38.000000,32.000000,146.00000
25%,44.750000,35.750000,155.00000
50%,55.500000,41.000000,160.50000
75%,62.750000,46.750000,163.50000
max,75.000000,62.000000,176.00000


In [13]:
from sklearn.model_selection import train_test_split
Unique_num=list(psm.matched_ids[psm.matched_ids['PT_ID'].isin(Unique_num)]['PT_ID'])
Positive=orig[orig['PT_ID'].isin(Unique_num)]
#print(Positive)
train_set, test_set = train_test_split(Positive, test_size=0.5, random_state=42)

test_set1, test_set2=train_test_split(test_set, test_size=0.5, random_state=42)
set1_labno=list(train_set['PT_ID'])
set1_labno.extend(list(psm.matched_ids[psm.matched_ids['PT_ID'].isin(set1_labno)]['matched_ID']))
set1=orig[orig['PT_ID'].isin(set1_labno)]
set1_data=data[data['PT_ID'].isin(set1_labno)]


set2_labno=list(test_set1['PT_ID'])
set2_labno.extend(list(psm.matched_ids[psm.matched_ids['PT_ID'].isin(set2_labno)]['matched_ID']))
set2=orig[orig['PT_ID'].isin(set2_labno)]
set2_data=data[data['PT_ID'].isin(set2_labno)]

set3_labno=list(test_set2['PT_ID'])
set3_labno.extend(list(psm.matched_ids[psm.matched_ids['PT_ID'].isin(set3_labno)]['matched_ID']))
set3=orig[orig['PT_ID'].isin(set3_labno)]
set3_data=data[data['PT_ID'].isin(set3_labno)]
#pd.DataFrame(set1).to_csv(r'set1.csv',index=False)

In [14]:
pd.DataFrame(set1).to_csv(r'set1_2.csv',index=False)
pd.DataFrame(set2).to_csv(r'set2_2.csv',index=False)
pd.DataFrame(set3).to_csv(r'set3_2.csv',index=False)

In [458]:
control=pd.read_csv('Controls.csv')
print(len(control))
random.seed(15722) 
control_list=random.sample(list(control['PT_ID']),24)
print(control_list)
pd.DataFrame(control_list).to_csv(r'Control_selected.csv',index=False)


195
['PT743', 'PT863', 'PT890', 'PT782', 'PT857', 'PT838', 'PT835', 'PT833', 'PT821', 'PT859', 'PT727', 'PT777', 'PT828', 'PT773', 'PT774', 'PT735', 'PT791', 'PT871', 'PT768', 'PT826', 'PT853', 'PT712', 'PT877', 'PT815']


In [17]:
from pandas.errors import DataError
import pandas_profiling
set3_positive=set3[set3['COVID Result']=='Positive']
#print(set1_positive)
set3_data_positive=set3_data[set3_data['COVID Result']==1]
set3_positive['BMI']=set3_positive['Weight']/((set3_positive['Height']/100)*(set3_positive['Height']/100))
#print(set2_positive)
#set1_positive['Height']=set1_data_positive['Height']
set3_positive.describe()
#pandas_profiling.ProfileReport(set3_positive)

/tmp/ipykernel_176299/3713969127.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set3_positive['BMI']=set3_positive['Weight']/((set3_positive['Height']/100)*(set3_positive['Height']/100))


,Age,Weight,Height,BMI
count,6.000000,6.000000,5.000000,5.000000
mean,57.333333,46.333333,161.000000,16.810454
std,15.292700,8.594572,1.581139,2.235579
min,37.000000,35.000000,159.000000,13.844389
25%,48.250000,41.500000,160.000000,15.055139
50%,56.500000,46.500000,161.000000,17.968750
75%,65.500000,49.250000,162.000000,18.132017
max,80.000000,60.000000,163.000000,19.051974


In [16]:

import pandas_profiling
set3_negative=set3[set3['COVID Result']=='Negative']
set3_data_negative=set3_data[set3_data['COVID Result']==0]
set3_negative['BMI']=set3_data_negative['BMI']
set3_negative['BMI']=set3_negative['Weight']/((set3_negative['Height']/100)*(set3_negative['Height']/100))
#set1_negative['Height']=set1_data_negative['Height']

#pandas_profiling.ProfileReport(set3_negative)
print(set3_negative.describe())

             Age     Weight      Height        BMI
count   6.000000   6.000000    3.000000   3.000000
mean   63.500000  40.166667  151.333333  16.267187
std     5.205766   6.735478    5.033223   2.980627
min    56.000000  34.000000  146.000000  14.381986
25%    60.500000  35.000000  149.000000  14.549026
50%    63.500000  38.500000  152.000000  14.716066
75%    67.250000  43.500000  154.000000  17.209788
max    70.000000  51.000000  156.000000  19.703509


/tmp/ipykernel_176299/2159370642.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set3_negative['BMI']=set3_data_negative['BMI']
/tmp/ipykernel_176299/2159370642.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set3_negative['BMI']=set3_negative['Weight']/((set3_negative['Height']/100)*(set3_negative['Height']/100))


In [18]:
df = pd.concat(map(pd.read_csv, ['set1_2.csv', 'set2_2.csv','set3_2.csv']))

In [19]:
import pandas_profiling
df_positive=df[df['COVID Result']=='Positive']
df_positive['BMI']=df_positive['Weight']/((df_positive['Height']/100)*(df_positive['Height']/100))
#pandas_profiling.ProfileReport(df_positive)

/tmp/ipykernel_176299/3829130080.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive['BMI']=df_positive['Weight']/((df_positive['Height']/100)*(df_positive['Height']/100))


In [20]:
import pandas_profiling
df_negative=df[df['COVID Result']=='Negative']
df_negative['Height'].fillna(mean, inplace=True)
df_negative['BMI']=df_negative['Weight']/((df_negative['Height']/100)*(df_negative['Height']/100))

print(df_negative)
#pandas_profiling.ProfileReport(df_negative)

    PT_ID  Age  Gender Socio Economic Status New or Previously treated  \
0   PT014   43    male                   BPL                       NEW   
1   PT083   49    male                   BPL                       NEW   
2   PT168   45    male                   BPL                       NEW   
3   PT313   75    male               UNKNOWN                       NEW   
4   PT319   55    male                   BPL                       NEW   
8   PT121   50  female                   BPL                       NEW   
10  PT064   42    male                   BPL                       NEW   
11  PT355   60  female                   BPL                       NEW   
15  PT137   61    male                   BPL                       NEW   
16  PT423   68    male               UNKNOWN                       NEW   
21  PT037   68    male                   BPL                       NEW   
22  PT027   44    male                   BPL                       NEW   
1   PT073   42    male                

/tmp/ipykernel_176299/3150977652.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative['Height'].fillna(mean, inplace=True)
/tmp/ipykernel_176299/3150977652.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative['BMI']=df_negative['Weight']/((df_negative['Height']/100)*(df_negative['Height']/100))


In [21]:
from tableone import TableOne
import pandas as pd
#data_sub=data
data_sub = pd.read_csv('Subset_19_11.csv')
data_sub['Treatment regimen']='Normal'
data_sub.shape

(48, 26)

In [22]:
columns=['Age', 'Gender',
       'Socio Economic Status', 'New or Previously treated', 'Weight',
       'Height', 'Treatment regimen', 'TREATMENT PHASE', 'Treatment outcome',
       'HIV status', 'Diabetes', 'TOBACCO-Smoking', 'Alcohol',
       'Rif Resistance', 'Smear', 'RIF(D,ND,I)', 'INH(InhA) (D,ND,I)',
       'INH(KatG) (D,ND,I)',
       'FQ class resistance (ND/D/I)', 'SLI (resi)', 'SLID (eis)',
        'COVID Result']

In [23]:
categorical=['Gender','Socio Economic Status', 'New or Previously treated','Treatment regimen', 'TREATMENT PHASE', 'Treatment outcome',
       'HIV status', 'Diabetes', 'TOBACCO-Smoking', 'Alcohol',
       'Rif Resistance', 'Smear', 'RIF(D,ND,I)', 'INH(InhA) (D,ND,I)',
       'INH(KatG) (D,ND,I)',
       'FQ class resistance (ND/D/I)', 'SLI (resi)', 'SLID (eis)',
        'COVID Result']
continuous=['Age','Weight','Height',]
groupby = 'COVID Result'


mytable = TableOne(data_sub, columns=columns, categorical=categorical, groupby=groupby,missing=False, pval=True)



In [24]:
print(mytable.tabulate(tablefmt="csv"))

                                                              Overall      Negative     Positive     P-Value
-----------------------------------  -----------------------  -----------  -----------  -----------  ---------
n                                                             48           24           24
Age, mean (SD)                                                53.6 (11.7)  55.2 (10.8)  52.0 (12.5)  0.334
Gender, n (%)                        female                   9 (18.8)     5 (20.8)     4 (16.7)     1.000
                                     male                     39 (81.2)    19 (79.2)    20 (83.3)
Socio Economic Status, n (%)         BPL                      39 (81.2)    21 (87.5)    18 (75.0)    0.421
                                     UNKNOWN                  8 (16.7)     3 (12.5)     5 (20.8)
                                     Unknown                  1 (2.1)                   1 (4.2)
New or Previously treated, n (%)     NEW                      48 (100.0)   24

In [430]:
mytable.to_csv('gender_data_seed.csv')